### Projeto comparação entre algoritmos e modelos de Computação evolutiva

Achando o diretório em que esse arquivo se encontra 

In [1]:
import sys
import os

# Obtém o caminho do diretório do script atual
script_dir = os.getcwd()

# Adiciona o diretório pai ao caminho do sistema
sys.path.append(script_dir)

In [2]:
import pygame # Nos repositórios de referência essa lib é sempre importada tendo em vista que o gymnasium (gym) utiliza a mesma, deve evitar problemas

import time # será utilizada para coletar o período temporal necessário para que algumas funções sejam executadas
import numpy as np # numpy por motivos óbvios  

import torch # Todas as redes neurais serão criados utilizando PyTorch
import torch.nn as nn
import torch.optim as optim

from redes_neurais.rede_MLP import MLP
from func_aux.auxiliares import *

import matplotlib.pyplot as plt # Demonstração de Gráficos e imagens 

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# usar GPU compatível com CUDA costuma ser mais rápido
if torch.cuda.is_available():
    is_gpu = torch.device("cuda")
else:
    is_gpu = torch.device("cpu")

torch.set_default_device(is_gpu)

In [4]:
import gymnasium as gym

In [5]:
def joga_jogo(env, rede_neural):

    if isinstance(env, str):
        env = gym.make(env)

    imagem, _ = env.reset()
    passos = 0
    game_recompenca = 0

    while True: # inicializa o loop do jogo
        passos += 1 # conta quantas vezes a IA foi requisitada ate o complecionismo do mesmo

        acao = escolhe_acao(rede_neural, imagem)

        nova_imagem, recompensa, finalizado, truncado, _ = env.step(acao)
        done = finalizado or truncado
        game_recompenca += recompensa

        imagem = nova_imagem

        if done:
            return rede_neural, passos, game_recompenca

In [6]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []
    recompencas = []
    contador_recompencas = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
        
    return selecao_final[0], obs

In [7]:
env = gym.make("CartPole-v1")
rede_melhor, obs = treinamento_Foda(
    env = env,
    rede_neural_hidden = [36,36],
    geracoes = 20,
    n_por_geracao = 30,
    n_selecionados = 6,
    pais = True,
    learning_rate = 0.1,
    objetivo = 500
)

TypeError: hidden_size should be of type int, got: list

In [ ]:
print(rede_melhor)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=4, out_features=36, bias=True)
    (1): ReLU()
    (2): Linear(in_features=36, out_features=36, bias=True)
    (3): ReLU()
    (4): Linear(in_features=36, out_features=2, bias=True)
  )
)
